In [1]:
import pandas as pd
import numpy as np
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
from biogeme.expressions import Beta
from biogeme.expressions import log
import math

In [2]:
df = pd.read_csv("data/us_estdata_cleaned.csv")
df

,Unnamed: 0.1,Unnamed: 0,RT,SERIALNO,DIVISION,SPORDER,PUMA,REGION,ST,ADJINC,...,PINCP_ORIG_ADJ,HINCP_ORIG_ADJ,SMOCP_ORIG_ADJ,GRNTP_ORIG_ADJ,WAGP_ORIG_ADJ,PERNP_ORIG_ADJ,UNEMP_PCT_ORIG_ADJ,OWN_JOB_ORIG,TYPE_ORIG,CBSA_NAME_ORIG
0,0,0,P,2018HU0637969,3,1,1104,2,17,1013097,...,40000.0,72300.0,944.0,913.0,36800.0,38000.0,0.036364,13058,0,218.0
1,1,1,P,2018HU0808038,5,5,505,3,24,1013097,...,51000.0,96400.0,1478.0,1310.0,47000.0,49700.0,0.045407,0,0,341.0
2,2,2,P,2018HU1051188,9,1,7310,4,6,1013097,...,43000.0,94200.0,1998.0,1620.0,37000.0,40000.0,0.045469,46774,0,13.0
3,3,3,P,2018HU0461416,2,1,4110,1,36,1013097,...,40000.0,90200.0,1840.0,1670.0,38000.0,40000.0,0.051709,18982,0,367.0
4,4,4,P,2018GQ0016994,5,1,51167,3,51,1013097,...,45000.0,90050.0,1558.0,1417.0,41150.0,42000.0,0.025789,13230,1,477.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253085,253085,253085,P,2018HU0435287,5,2,1400,3,24,1013097,...,40000.0,70700.0,1061.0,995.0,32800.0,35000.0,0.057456,2551,1,321.0
253086,253086,253086,P,2018HU0652247,8,2,900,4,8,1013097,...,40000.0,68500.0,933.0,940.0,30000.0,35000.0,0.035656,6684,3,-1.0
253087,253087,253087,P,2018HU0192642,8,1,824,4,8,1013097,...,50000.0,95100.0,1545.0,1410.0,45000.0,48000.0,0.029020,116203,3,-1.0
253088,253088,253088,P,2018HU0750302,7,1,5915,3,48,1013097,...,37000.0,69420.0,1004.0,1039.0,32800.0,34700.0,0.040172,30030,0,89.0


In [3]:
NUM_ALTERNATIVES = 100

In [4]:
# reading in external data
puma_acs_data = pd.read_csv("data/ACS_2018.csv")
puma_lodes_data = pd.read_csv("data/wac_puma.csv")

puma_acs_data = puma_acs_data.set_index("PUMA")
puma_lodes_data = puma_lodes_data.set_index("puma")

# puma_acs_data = puma_acs_data[~puma_acs_data["Geo_STATE"].isin([2, 15, 72])]

# puma_acs_data.index = puma_recodes.loc[puma_acs_data.index].values.reshape((-1))
# puma_lodes_data.index = puma_recodes.loc[puma_lodes_data.index].values.reshape((-1))

puma_acs_data.head()

,Geo_FIPS,Geo_GEOID,Geo_NAME,Geo_QName,Geo_STUSAB,Geo_LOGRECNO,Geo_STATE,Geo_PUMA5,Total Population,Population Density (Per Sq. Mile),...,Civilian Population 18 Years and Over Veteran,Civilian Population 18 Years and Over Veteran 18 to 64 Years,Civilian Population 18 Years and Over Veteran 65 Years and Over,Civilian Population 18 Years and Over Nonveteran,Civilian Population 18 Years and Over Nonveteran 18 to 64 Years,Civilian Population 18 Years and Over Nonveteran 65 Years and Over,Total Population Native Born,Total Population Foreign Born,Total Population Foreign Born Naturalized Citizen,Total Population Foreign Born Not a Citizen
PUMA,,,,,,,,,,,,,,,,,,,,,
100100,100100,79500US0100100,"Lauderdale, Colbert, Franklin & Marion (Northe...","Lauderdale, Alabama",al,57,1,100,184923,223.027706,...,9722,4332,5390,136809,105950,30859,178843.0,6080.0,1312.0,4768.0
100200,100200,79500US0100200,Limestone & Madison (Outer) Counties--Huntsvil...,Limestone & Madison (Outer) Counties--Huntsvil...,al,58,1,200,196618,237.132544,...,16167,9728,6439,135089,114171,20918,190579.0,6039.0,3440.0,2599.0
100301,100301,79500US0100301,Huntsville (North) & Madison (East) Cities PUM...,Huntsville (North) & Madison (East) Cities PUM...,al,59,1,301,133112,1459.688320,...,12813,8417,4396,91051,75831,15220,123801.0,9311.0,4154.0,5157.0
100302,100302,79500US0100302,"Huntsville City (Central & South) PUMA, Alabama","Huntsville City (Central & South) PUMA, Alabama",al,60,1,302,101737,1634.875764,...,8523,5271,3252,71585,57043,14542,95354.0,6383.0,3075.0,3308.0
100400,100400,79500US0100400,"DeKalb & Jackson Counties PUMA, Alabama","DeKalb & Jackson Counties PUMA, Alabama",al,61,1,400,123121,148.490962,...,5845,2846,2999,89462,70118,19344,117038.0,6083.0,669.0,5414.0


In [5]:
# creating some new fields for PUMAs
puma_acs_data["UNEMP"] = puma_acs_data["Population 16 Years and Over in Labor Force Civilian Unemployed"] / puma_acs_data["Population 16 Years and Over in Labor Force Civilian"]
puma_acs_data["COLLEGE"] = puma_acs_data["Population 3 Years and Over Enrolled in School Private School College"] + puma_acs_data["Population 3 Years and Over Enrolled in School Public School College"]
puma_acs_data["VACANCY_PCT"] = puma_acs_data["Housing Units Vacant"] / puma_acs_data["Housing Units"]

In [6]:
puma_acs_data["COLLEGE"] = puma_acs_data["COLLEGE"].fillna(0)
puma_acs_data["Total Population Foreign Born"] = puma_acs_data["Total Population Foreign Born"].fillna(0)

In [7]:
# reading in MIGPUMA-PUMA distance data
distances = pd.read_csv("data/puma_distance_matrix_fixed.csv").set_index("Unnamed: 0")
distances.index = distances.index.astype(int)
distances.columns = distances.columns.astype(int)
# distances.columns = puma_recodes.loc[distances.columns.astype(int)].values.reshape((-1))
distances

,600105,600102,608502,600108,600107,600101,600110,608504,600109,600106,...,2602200,2600900,4702402,4702300,4702401,4700500,4702501,5310400,5310300,5310100
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
600100,2.811224e+04,2.811224e+04,39057.0,2.811224e+04,2.811224e+04,2.811224e+04,2.811224e+04,32785.0,2.811224e+04,2.811224e+04,...,3659560.0,3658028.0,3704534.0,3707792.0,3697348.0,3709156.0,3675491.0,1510819.0,1366000.0,1521187.0
600700,2.676500e+05,2.541540e+05,308889.0,2.933460e+05,2.845210e+05,2.477670e+05,2.743320e+05,302616.0,2.903600e+05,2.738470e+05,...,3548479.0,3546947.0,3757628.0,3748449.0,3750726.0,3741417.0,3716148.0,1293877.0,1149057.0,1304245.0
601100,3.420630e+05,3.285670e+05,383302.0,3.677590e+05,3.589340e+05,3.221800e+05,3.618530e+05,377029.0,3.647730e+05,3.482590e+05,...,3646878.0,3645346.0,3856026.0,3846847.0,3849125.0,3839816.0,3814546.0,1246358.0,1101538.0,1256725.0
601300,5.418500e+04,4.434600e+04,90212.0,8.001100e+04,6.752000e+04,4.544600e+04,6.876300e+04,83940.0,7.168300e+04,5.517000e+04,...,3635231.0,3633699.0,3743085.0,3746342.0,3735899.0,3747707.0,3714042.0,1486490.0,1341670.0,1496858.0
601500,4.872780e+05,4.737820e+05,528517.0,5.129740e+05,5.041490e+05,4.673950e+05,5.070680e+05,522244.0,5.099870e+05,4.934740e+05,...,3594176.0,3592644.0,3803325.0,3794145.0,3796423.0,3787114.0,3761844.0,1149082.0,1004262.0,1159450.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304007,3.996091e+06,4.004766e+06,3970737.0,3.980928e+06,3.987472e+06,4.012807e+06,3.958553e+06,3961958.0,3.991797e+06,3.985390e+06,...,1210028.0,1269788.0,373179.0,429406.0,391176.0,457262.0,418124.0,4143960.0,4251644.0,4527073.0
5151000,4.385759e+06,4.394434e+06,4360405.0,4.370596e+06,4.377141e+06,4.402476e+06,4.348221e+06,4351626.0,4.381466e+06,4.375058e+06,...,917732.0,1041456.0,718843.0,663499.0,695569.0,705881.0,692139.0,4231226.0,4338910.0,4661344.0
5151001,4.543694e+06,4.530198e+06,4542272.0,4.569390e+06,4.560565e+06,4.523811e+06,4.547511e+06,4533493.0,4.566404e+06,4.549890e+06,...,1050887.0,1169711.0,900710.0,845366.0,877436.0,887748.0,874006.0,4350356.0,4458040.0,4780474.0


In [8]:
df["CHOICE"] = np.where(df["STAY"] == 1, 0, df["CHOSEN"])

In [9]:
puma_recodes = pd.factorize(sorted(df["CHOSEN"].unique()))[1]
puma_recodes = pd.DataFrame(index=puma_recodes, data=np.arange(1, 2337), columns=["code"])
puma_recodes

,code
100100,1
100200,2
100301,3
100302,4
100400,5
...,...
5600100,2332
5600200,2333
5600300,2334
5600400,2335


In [10]:
cbsas = pd.read_csv("data/puma_density.csv")
cbsas = cbsas.set_index("GEOID")
cbsas["type_num"] = 2
cbsas["type_num"] = np.where(cbsas["type"] == "T34", 0, cbsas["type_num"])
cbsas["type_num"] = np.where(cbsas["type"] == "Metro", 1, cbsas["type_num"])
# cbsas["type_num"] = np.where(cbsas["type"] == "Micro", 2, cbsas["type_num"])
cbsas["Name_num"] = pd.factorize(cbsas["CBSA_name"])[0]
cbsas

,Unnamed: 0,GISMATCH,GISJOIN,STATEFIP,State,PUMA,Name,center,type,CBSA_name,type_num,Name_num
GEOID,,,,,,,,,,,,
600105,0,600105,G06000105,6,California,105,"Alameda County (West)--San Leandro, Alameda & ...",POINT (-2258808.793332302 339396.4722123478),T34,"San Francisco-Oakland-Hayward, CA Metro Area",0,0
600102,1,600102,G06000102,6,California,102,Alameda County (Northwest)--Oakland (Northwest...,POINT (-2262119.20387295 349529.9544024405),T34,"San Francisco-Oakland-Hayward, CA Metro Area",0,0
608502,2,608502,G06008502,6,California,8502,Santa Clara County (Northwest)--Sunnyvale & Sa...,POINT (-2251625.280558518 299700.2119385958),Metro,"San Jose-Sunnyvale-Santa Clara, CA Metro Area",1,1
600108,3,600108,G06000108,6,California,108,"Alameda County (Southwest)--Union City, Newark...",POINT (-2249381.571782368 314454.5304018197),T34,"San Francisco-Oakland-Hayward, CA Metro Area",0,0
600107,4,600107,G06000107,6,California,107,Alameda County (Central)--Hayward City PUMA,POINT (-2249209.156361292 325491.2460234718),T34,"San Francisco-Oakland-Hayward, CA Metro Area",0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4700500,2373,4700500,G47000500,47,Tennessee,500,Sumner County--Hendersonville City PUMA,POINT (845555.1346444382 -73006.03095774697),Metro,"Nashville-Davidson--Murfreesboro--Franklin, TN...",1,283
4702501,2374,4702501,G47002501,47,Tennessee,2501,Nashville-Davidson (East) PUMA,POINT (834713.9667902931 -111417.8817361052),Metro,"Nashville-Davidson--Murfreesboro--Franklin, TN...",1,283
5310400,2375,5310400,G53010400,53,Washington,10400,"Stevens, Okanogan, Pend Oreille & Ferry Counti...",POINT (-1684664.90787742 1429380.26889712),NaN,NaN,2,-1


In [11]:
# reading in PUMA-MIGPUMA equivalency data
puma_migpuma = pd.read_excel("data/puma_migpuma.xlsx", dtype="str")
puma_migpuma["PUMA"] = puma_migpuma["State"] + puma_migpuma["PUMA"]
puma_migpuma["MIGPUMA"] = puma_migpuma["State"] + puma_migpuma["MIGPUMA"]
# some MIGPUMA fields have footnotes
puma_migpuma["MIGPUMA"] = puma_migpuma["MIGPUMA"].str.strip("*")
puma_migpuma = puma_migpuma.set_index("PUMA")
# cleaning up puma-migpuma equivalency files
puma_migpuma = puma_migpuma.dropna()
puma_migpuma["State"] = puma_migpuma["State"].astype(int)
puma_migpuma.index = puma_migpuma.index.astype(int)
puma_migpuma.head()

,State,MIGPUMA
PUMA,,
100100,1,0100190
100200,1,0100290
100301,1,0100290
100302,1,0100290
100400,1,0100400


In [12]:
leave_puma = pd.read_csv("data/leave_puma.csv").set_index("CHOSEN")
leave_puma["UNEMP_PCT"] = leave_puma["UNEMP"] / (leave_puma["EMP"] + leave_puma["UNEMP"])
leave_puma

,JWMNP,FINCP,GRNTP,HINCP,SMOCP,PINCP,HH_COST,PERNP,WAGP,UNEMP,EMP,type,UNEMP_PCT
CHOSEN,,,,,,,,,,,,,
100100,13.0,48000.0,680.0,33550.0,753.0,21800.0,0.262523,21800.0,20500.0,3,83,leavers,0.034884
100200,20.0,70000.0,785.0,70000.0,1107.0,35000.0,0.162381,34000.0,34000.0,3,86,leavers,0.033708
100301,15.0,77400.0,973.5,63600.0,977.0,30000.0,0.175737,30000.0,28600.0,8,77,leavers,0.094118
100302,15.0,62800.0,920.0,55000.0,1131.0,30000.0,0.217411,29050.0,28150.0,6,92,leavers,0.061224
100400,21.5,30300.0,580.0,35600.0,775.0,28600.0,0.159860,27000.0,27000.0,6,43,leavers,0.122449
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5600100,10.0,76000.0,1005.0,55900.0,1001.0,26000.0,0.180750,26000.0,25000.0,3,84,leavers,0.034483
5600200,5.0,54000.0,922.0,61000.0,1130.0,35000.0,0.193548,25000.0,25000.0,2,41,leavers,0.046512
5600300,10.0,63700.0,930.0,42620.0,1338.0,20000.0,0.277500,19000.0,18000.0,12,135,leavers,0.081633


In [13]:
import random

In [14]:
alts = []
for puma in puma_recodes.index:
    puma = int(puma)
    key = f"ALT{puma}_available"
    alts.append(pd.DataFrame(0, index=df.index, columns=[key]))

In [15]:
df_alts = pd.concat([df, pd.concat(alts, axis=1)], axis=1)
df_alts

In [16]:
from tqdm import tqdm

In [17]:
# sampling NUM_ALTERNATIVES pumas for each individual
random.seed(5063)
for i, row in tqdm(df_alts.iterrows(), total=df_alts.shape[0]):
    stay = row["STAY"]
    chosen = 0
    # this list already excludes Hawaii/Alaska/Puerto Rico
    li = puma_recodes.index.values
    if stay == 0:
        chosen = row["CHOSEN"]
        li = li[li != chosen]
    pumas = random.sample(list(li), NUM_ALTERNATIVES)
    if stay == 0:
        pumas[0] = chosen
    indexes = np.empty(NUM_ALTERNATIVES, dtype=object)
    for j, puma in enumerate(pumas):
        puma = int(puma)
        key = f"ALT{puma}_available"
        indexes[j] = key
    df_alts.loc[i, indexes] = 1

In [18]:
# df_alts.to_csv("availability_finished_estdata.csv")

In [19]:
# df_alts = pd.read_csv("availability_finished_estdata.csv")

In [20]:
dists = []
for puma in tqdm(puma_recodes.index):
    puma = int(puma)
    key = f"ALT{puma}_dist"
    curr = distances.loc[df_alts["ORIGIN"], puma].values
    dists.append(pd.DataFrame(curr, index=df.index, columns=[key]))

100%|██████████| 2336/2336 [00:15<00:00, 150.16it/s]


In [21]:
df_alts = df_alts.iloc[:len(df_alts)]

In [22]:
df_alts = pd.merge(df_alts, df[["NAICS"]], left_index=True, right_index=True)

In [23]:
own_job = []
for puma in tqdm(puma_recodes.index):
    puma = int(puma)
    key = f"ALT{puma}_own_job"
    jobs = np.where(df_alts["NAICS"].isna(), 0, puma_lodes_data.loc[puma, df["NAICS"].fillna("EDU")])
    own_job.append(pd.DataFrame(jobs, columns=[key], index=df.index))

100%|██████████| 2336/2336 [01:57<00:00, 19.81it/s]


In [24]:
df_full = pd.concat([df_alts, pd.concat(dists, axis=1), pd.concat(own_job, axis=1)], axis=1)

In [25]:
df_full["MED"] = np.where(df_full["NAICS"] == "MED", 1, 0)
df_full["MFG"] = np.where(df_full["NAICS"] == "MFG", 1, 0)
df_full["RET"] = np.where(df_full["NAICS"] == "RET", 1, 0)
df_full["EDU"] = np.where(df_full["NAICS"] == "EDU", 1, 0)
df_full["ADM"] = np.where(df_full["NAICS"] == "ADM", 1, 0)
df_full["FOD"] = np.where(df_full["NAICS"] == "FOD", 1, 0)
df_full["PRF"] = np.where(df_full["NAICS"] == "PRF", 1, 0)
df_full["TRN"] = np.where(df_full["NAICS"] == "TRN", 1, 0)
df_full["SRV"] = np.where(df_full["NAICS"] == "SRV", 1, 0)
df_full["FIN"] = np.where(df_full["NAICS"] == "FIN", 1, 0)
df_full["WHL"] = np.where(df_full["NAICS"] == "WHL", 1, 0)
df_full["AGR"] = np.where(df_full["NAICS"] == "AGR", 1, 0)
df_full["PUB"] = np.where(df_full["NAICS"] == "PUB", 1, 0)
df_full["INF"] = np.where(df_full["NAICS"] == "INF", 1, 0)
df_full["ENT"] = np.where(df_full["NAICS"] == "ENT", 1, 0)
df_full["REL"] = np.where(df_full["NAICS"] == "REL", 1, 0)
df_full["UTL"] = np.where(df_full["NAICS"] == "UTL", 1, 0)
df_full["EXT"] = np.where(df_full["NAICS"] == "EXT", 1, 0)
df_full["MNG"] = np.where(df_full["NAICS"] == "MNG", 1, 0)
df_full["CON"] = np.where(df_full["NAICS"] == "CON", 1, 0)
df_full["NO_IND"] = np.where(df_full["NAICS"].isna(), 1, 0)

df_full["IN_COLLEGE"] = np.where((df_full["SCHG"] == 15) | (df_full["SCHG"] == 16), 1, 0)

df_full["AGE_18_34"] = np.where(df_full["AGEP"] <= 34, 1, 0)
df_full["AGE_35_64"] = np.where((df_full["AGEP"] >= 35) & (df_full["AGEP"] <= 64), 1, 0)
df_full["AGE_OVER_65"] = np.where((df_full["AGEP"] >= 65), 1, 0)
df_full["FOREIGN"] = np.where(df_full["NATIVITY"] == 2, 1, 0)

df_full["AGE_18_22"] = np.where(df_full["AGEP"] <= 22, 1, 0)
df_full["AGE_23_29"] = np.where((df_full["AGEP"] >= 23) & (df_full["AGEP"] <= 29), 1, 0)
df_full["AGE_30_39"] = np.where((df_full["AGEP"] >= 30) & (df_full["AGEP"] <= 39), 1, 0)
df_full["AGE_40_49"] = np.where((df_full["AGEP"] >= 40) & (df_full["AGEP"] <= 49), 1, 0)
df_full["AGE_50_64"] = np.where((df_full["AGEP"] >= 50) & (df_full["AGEP"] <= 64), 1, 0)

df_full["EDU_LESS_HIGH"] = np.where(df_full["SCHL"] <= 15, 1, 0)
df_full["EDU_HIGH"] = np.where((df_full["SCHL"] <= 20) & (df_full["SCHL"] >= 16), 1, 0) 
df_full["EDU_COLLEGE"] = np.where(df_full["SCHL"] >= 21, 1, 0)
df_full["EDU_NOCOLLEGE"] = np.where(df_full["EDU_COLLEGE"] == 0, 1, 0)

df_full["WOMAN_CHILD"] = np.where((df_full["PAOC"] >= 1) & (df_full["PAOC"] <= 3), 1, 0)
df_full["UNEMPLOYED"] = np.where(df_full["ESR"] == 3, 1, 0)

df_full["MALE"] = np.where(df_full["SEX"] == 1, 1, 0)
df_full["FEMALE"] = np.where(df_full["SEX"] == 0, 1, 0)

df_full["MARRIED"] = np.where(df_full["MAR"] == 1, 1, 0)

# df_full["child_old"] = np.where(df_full["child"] == df_full["REC_CHILD"], 0, df_full["child"])
# df_full["child_old"].value_counts()
df_full["child"].value_counts()

df_full["REC_NO_MAR"] = np.where((df_full["MARHD"] == 1) | (df_full["MARHW"] == 1), 1, 0)
df_full["REC_NO_MAR"].value_counts()

df_full["MARHM_new"] = np.where(df_full["MARHM"] == 2, 0, df_full["MARHM"])
df_full["MARHM_new"].value_counts()

df_full["married_old"] = np.where((df_full["MARHM"] == df_full["MARRIED"]), 0, df_full["MARRIED"])
df_full["married_old"].value_counts()

df_full["MILITARY"] = np.where(df_full["MIL"] == 1, 1, 0)

# up to debate still
df_full["AGR_EXT"] = np.where((df_full["AGR"] == 1) | (df_full["EXT"] == 1), 1, 0)
df_full["HIGH_ED"] = np.where((df_full["MED"] == 1) | (df_full["EDU"] == 1) | (df_full["PRF"] == 1) | (df_full["MED"] == 1) | (df_full["FIN"] == 1) | (df_full["INF"] == 1) | (df_full["MED"] == 1), 1, 0)
df_full["LICENSE"] = np.where((df_full["SRV"] == 1) | (df_full["REL"] == 1), 1, 0)
df_full["OTHER_JOB"] = np.where((df_full["AGR_EXT"] == 0) & (df_full["HIGH_ED"] == 0) & (df_full["LICENSE"] == 0) & (df_full["NO_IND"] == 0), 1, 0)

df_full["MICRO_adj_ORIG"] = np.where((df_full["TYPE_ORIG"] == 3) | (df_full["TYPE_ORIG"] == 2), 1, 0)
df_full["METRO"] = np.where(df_full["TYPE_ORIG"] == 1, 1, 0)
df_full["T34"] = np.where(df_full["TYPE_ORIG"] == 0, 1, 0)

df_full["UNEMPLOYED"] = np.where((df_full["ESR"] == 3) | (df_full["ESR"] == 6), 1, 0)
df_full["IN_LF"] = np.where(df_full["ESR"] == 6, 0, 1)

df_full["WORK2_MAR"] = np.where(df_full["FES"] == 1, 1, 0)
df_full["WORK1_MAR"] = np.where((df_full["FES"] <= 4) & (df_full["FES"] >= 2), 1, 0)
df_full["OTHER_FAMILY"] = np.where((df_full["HHT"] == 2) | (df_full["HHT"] == 3), 1, 0)

df_full.loc[df_full["CBSA_NAME_ORIG"] == -1, "CBSA_NAME_ORIG"] = -2

df_full["MIGSP_ORIG"] = df_full["ORIGIN"].astype(str).str.zfill(7).str[0:2].astype(int)

In [26]:
# clean up the database (Biogeme Database can only have numerical values)
df_full = df_full.select_dtypes(['number'])
df_full = df_full.fillna(0)

In [27]:
# making the Biogeme Database that is used for the model estimation
database = db.Database('us_data', df_full)

# The following statement allows you to use the names of the
# variable as Python variable. (in the utility functions)
globals().update(database.variables)

In [28]:
# Staying Choice Parameters to be Estimated
c_stay_age_18_22 = Beta("c_stay_age_18_22", -0.4297984, None, None, 0)
c_stay_age_23_29 = Beta("c_stay_age_23_29", -0.572291, None, None, 0)
c_stay_age_30_39 = Beta("c_stay_age_30_39", -0.337318, None, None, 0)
c_stay_age_40_49 = Beta("c_stay_age_40_49", 0, None, None, 1)
c_stay_age_50_64 = Beta("c_stay_age_50_64", 0.1951023, None, None, 0)
c_stay_age_65 = Beta("c_stay_age_65", 0.6659279, None, None, 0)
c_stay_child = Beta("c_stay_child", -0.49784, None, None, 0)
c_stay = Beta("c_stay", 15.7542, None, None, 0)
c_stay_foreign = Beta("c_stay_foreign", 0.1775434, None, None, 0)
c_stay_dens = Beta("c_stay_dens", 0.00000437, None, None, 0)
c_stay_college = Beta("c_stay_college", 0.7150696, None, None, 0)
c_stay_rec_child = Beta("c_stay_rec_child", 0.05054558, None, None, 0)
c_stay_rec_mar = Beta("c_stay_rec_mar", -1.138993, None, None, 0)
c_stay_rec_nomar = Beta("c_stay_rec_nomar", -0.5003451, None, None, 0)
c_stay_mil = Beta("c_stay_mil", -0.79351627, None, None, 0)
c_stay_2work_mar = Beta("c_stay_2work_mar", 1.020378, None, None, 0)
c_stay_1work_mar = Beta("c_stay_1work_mar", 0.8788182, None, None, 0)
c_stay_otherfamily = Beta("c_stay_otherfamily", 0.66886, None, None, 0)
c_stay_ownjob = Beta("c_stay_ownjob", 0.8845035, None, None, 0)
c_stay_rentcost = Beta("c_stay_rentcost", -0.0009956133, None, None, 0)
c_stay_owncost = Beta("c_stay_owncost", 0.00040436, None, None, 0)
# c_stay_unemp_rate = Beta("c_stay_unemp_rate", 0, None, 0, 0)
c_stay_hh_val = Beta("c_stay_hh_val", 0.000001735451, None, None, 0)
c_stay_income = Beta("c_stay_income", 0.0000009994866, None, None, 0)

c_stay_T34 = Beta("c_stay_T34", 0.4236324, None, None, 0)
c_stay_metro = Beta("c_stay_metro", 0, None, None, 0)
c_stay_micro = Beta("c_stay_micro", -0.0694856, None, None, 0)
c_stay_edu_college = Beta("c_stay_edu_college", -0.2157945, None, None, 0)
c_stay_edu_high = Beta("c_stay_edu_high", 0, None, None, 0)
c_stay_edu_nohigh = Beta("c_stay_edu_nohigh", 0.04565692, None, None, 0)

In [29]:
# defining the staying utility function
V0 = c_stay + \
c_stay_age_18_22 * AGE_18_22 + c_stay_age_23_29 * AGE_23_29 + c_stay_age_30_39 * AGE_30_39 + c_stay_age_40_49 * AGE_40_49 + c_stay_age_50_64 * AGE_50_64 + c_stay_age_65 * AGE_OVER_65 \
+ c_stay_child * child + c_stay_rec_child * REC_CHILD \
+ c_stay_rec_mar * MARHM_new + c_stay_rec_nomar * REC_NO_MAR \
+ MARRIED * (c_stay_2work_mar * WORK2_MAR + c_stay_1work_mar * WORK1_MAR) + c_stay_otherfamily * OTHER_FAMILY \
+ c_stay_college * IN_COLLEGE + c_stay_mil * MILITARY + c_stay_ownjob * OWN_JOB_ORIG / TOT_JOBS_ORIG \
+ c_stay_hh_val * HH_MED_VAL_ORIG + (c_stay_rentcost * GRNTP_ORIG_ADJ + c_stay_owncost * SMOCP_ORIG_ADJ) \
+ c_stay_foreign * FOREIGN \
+ c_stay_edu_college * EDU_COLLEGE + c_stay_edu_nohigh * EDU_LESS_HIGH \
+ IN_LF * (c_stay_income * PINCP_ORIG_ADJ) \
+ c_stay_T34 * T34 + c_stay_micro * MICRO_adj_ORIG

In [30]:
V0

(((((((((((((((((((((((c_stay(15.7542) + (c_stay_age_18_22(-0.4297984) * AGE_18_22)) + (c_stay_age_23_29(-0.572291) * AGE_23_29)) + (c_stay_age_30_39(-0.337318) * AGE_30_39)) + (c_stay_age_40_49(0) * AGE_40_49)) + (c_stay_age_50_64(0.1951023) * AGE_50_64)) + (c_stay_age_65(0.6659279) * AGE_OVER_65)) + (c_stay_child(-0.49784) * child)) + (c_stay_rec_child(0.05054558) * REC_CHILD)) + (c_stay_rec_mar(-1.138993) * MARHM_new)) + (c_stay_rec_nomar(-0.5003451) * REC_NO_MAR)) + (MARRIED * ((c_stay_2work_mar(1.020378) * WORK2_MAR) + (c_stay_1work_mar(0.8788182) * WORK1_MAR)))) + (c_stay_otherfamily(0.66886) * OTHER_FAMILY)) + (c_stay_college(0.7150696) * IN_COLLEGE)) + (c_stay_mil(-0.79351627) * MILITARY)) + ((c_stay_ownjob(0.8845035) * OWN_JOB_ORIG) / TOT_JOBS_ORIG)) + (c_stay_hh_val(1.735451e-06) * HH_MED_VAL_ORIG)) + ((c_stay_rentcost(-0.0009956133) * GRNTP_ORIG_ADJ) + (c_stay_owncost(0.00040436) * SMOCP_ORIG_ADJ))) + (c_stay_foreign(0.1775434) * FOREIGN)) + (c_stay_edu_college(-0.2157945) *

In [31]:
# Destination Choice Parameters to be estimated
# Beta(name of the factor, initial value of the coefficient, lower bound, upper bound, whether or not
c_move = Beta("c_move", -0.1706511, None, None, 0)
c_destchoice_dist=Beta('c_destchoice_dist', -0.0000002684104, None, None, 0)
c_destchoice_logdist=Beta('c_destchoice_logdist', -0.3057492, None, None, 0)
c_destchoice_unemp_rate = Beta("c_destchoice_unemp_rate", -2.419098, None, None, 0)
c_destchoice_hhcost = Beta("c_destchoice_hhcost", -2.2776849, None, None, 0)
c_destchoice_college = Beta("c_destchoice_college", 0.00006571441, None, None, 0)
c_destchoice_age_18_34 = Beta("c_destchoice_age_18_34", 3.996802, None, None, 0)
c_destchoice_age_35_64 = Beta("c_destchoice_age_35_64", 1.517685, None, None, 0)
c_destchoice_age_over_65 = Beta("c_destchoice_age_over_65", 4.233292, None, None, 0)
c_destchoice_foreign = Beta("c_destchoice_foreign", 0.7225565, None, None, 0)
c_destchoice_entscore_18_34 = Beta("c_destchoice_entscore_18_34", 0.5341137, None, None, 0)
c_destchoice_entscore_35_64 = Beta("c_destchoice_entscore_35_64", 2.761052, None, None, 0)
c_destchoice_entscore_65 = Beta("c_destchoice_entscore_65", 2.096477, None, None, 0)
c_destchoice_military = Beta("c_destchoice_military", 0.0002028093, None, None, 0)

c_destchoice_T34_T34 = Beta("c_destchoice_T34_T34", 0, None, None, 1) 
c_destchoice_T34_Metro = Beta('c_destchoice_T34_Metro', 0.7080241, None, None, 0)
c_destchoice_T34_Micro = Beta("c_destchoice_T34_Micro", 0.8975212, None, None, 0)
c_destchoice_Metro_T34 = Beta("c_destchoice_Metro_T34", 0.2369336, None, None, 0)
c_destchoice_Metro_Metro = Beta('c_destchoice_Metro_Metro', 0.8342382, None, None, 0)
c_destchoice_Metro_Micro = Beta('c_destchoice_Metro_Micro', 1.012737, None, None, 0)
c_destchoice_Micro_T34 = Beta("c_destchoice_Micro_T34", --0.4684013, None, None, 0)
c_destchoice_Micro_Metro = Beta("c_destchoice_Micro_Metro", 0.5166342, None, None, 0)
c_destchoice_Micro_Micro = Beta("c_destchoice_Micro_Micro", 1.416916, None, None, 0)

c_destchoice_geo_spec_job = Beta("c_destchoice_geo_spec_job", 5.945615, None, None, 0)
c_destchoice_high_ed_job = Beta("c_destchoice_high_ed_job", 1.918109, None, None, 0)
c_destchoice_license_job = Beta("c_destchoice_license_job", 5.042017, None, None, 0)
c_destchoice_other = Beta("c_destchoice_other", 2.561332, None, None, 0)

c_destchoice_samestate = Beta("c_destchoice_samestate", 2.228781, None, None, 0)
c_destchoice_birthstate = Beta("c_destchoice_birthstate", 0.3655848, None, None, 0)
c_destchoice_samecbsa = Beta("c_destchoice_samecbsa", -0.05823393, None, None, 0)
c_destchoice_cbsa_dist = Beta("c_destchoice_cbsa_dist", -0.00002960449, None, None, 0)

In [32]:
for puma in puma_recodes.index:
    puma = int(puma)
    init = "V{0} = c_move + log(int(puma_acs_data.loc[puma, 'Total Population'])) + (CBSA_NAME_ORIG != int(cbsas.loc[puma, 'Name_num'])) * (c_destchoice_dist * ALT{0}_dist + c_destchoice_logdist * log(ALT{0}_dist + 1)) + (CBSA_NAME_ORIG == int(cbsas.loc[puma, 'Name_num'])) * c_destchoice_cbsa_dist * ALT{0}_dist + \
        \
    c_destchoice_hhcost * int(leave_puma.loc[puma, 'HH_COST']) + c_destchoice_college * IN_COLLEGE * int(puma_acs_data.loc[puma, 'COLLEGE']) + c_destchoice_foreign * FOREIGN * int(puma_acs_data.loc[puma, 'Total Population Foreign Born']) / int(puma_acs_data.loc[puma, 'Total Population']) + \
        \
    c_destchoice_age_18_34 * AGE_18_34 * int(puma_acs_data.loc[puma, 'Total Population 18 to 34 Years']) / int(puma_acs_data.loc[puma, 'Total Population']) + c_destchoice_age_35_64 * AGE_35_64 * int(puma_acs_data.loc[puma, 'Total Population 35 to 64 Years']) / int(puma_acs_data.loc[puma, 'Total Population']) + c_destchoice_age_over_65 * AGE_OVER_65 * int(puma_acs_data.loc[puma, 'Total Population 65 and Over']) / int(puma_acs_data.loc[puma, 'Total Population']) + \
        \
    int(puma_lodes_data.loc[puma, 'ENT']) / int(puma_lodes_data.loc[puma, 'TOT_JOBS']) * (c_destchoice_entscore_18_34 * AGE_18_34 + c_destchoice_entscore_35_64 * AGE_35_64 + c_destchoice_entscore_65 * AGE_OVER_65) + \
        \
    IN_LF * (c_destchoice_unemp_rate * float(leave_puma.loc[puma, 'UNEMP_PCT']) ) + \
        \
    (c_destchoice_T34_T34 * T34 + c_destchoice_Metro_T34 * METRO +  c_destchoice_Micro_T34 * MICRO_adj_ORIG) * (int(cbsas.loc[puma, 'type_num']) == 0)+ \
    (c_destchoice_T34_Metro * T34 + c_destchoice_Metro_Metro * METRO + c_destchoice_Micro_Metro * MICRO_adj_ORIG) * (int(cbsas.loc[puma, 'type_num']) == 1) + \
    (c_destchoice_T34_Micro * T34  + c_destchoice_Metro_Micro * METRO + c_destchoice_Micro_Micro * MICRO_adj_ORIG) * (int(cbsas.loc[puma, 'type_num']) == 2) + \
        \
    (c_destchoice_geo_spec_job * AGR_EXT + c_destchoice_high_ed_job * HIGH_ED + c_destchoice_license_job * LICENSE + c_destchoice_other * OTHER_JOB) * ALT{0}_own_job / int(puma_lodes_data.loc[puma, 'TOT_JOBS']) + c_destchoice_military * MILITARY * int(puma_acs_data.loc[puma, 'Population 16 Years and Over in Labor Force in Armed Forces']) + \
        \
    c_destchoice_birthstate * (POBP == int(puma_migpuma.loc[puma, 'State'])) + c_destchoice_samestate * (MIGSP_ORIG == int(puma_migpuma.loc[puma, 'State']))".format(puma)
    exec(init)

In [33]:
V100100

((((((((((((((((((c_move(-0.1706511) + log(`184923`)) + ((CBSA_NAME_ORIG != `47`) * ((c_destchoice_dist(-2.684104e-07) * ALT100100_dist) + (c_destchoice_logdist(-0.3057492) * log((ALT100100_dist + `1`)))))) + (((CBSA_NAME_ORIG == `47`) * c_destchoice_cbsa_dist(-2.960449e-05)) * ALT100100_dist)) + (c_destchoice_hhcost(-2.2776849) * `0`)) + ((c_destchoice_college(6.571441e-05) * IN_COLLEGE) * `10069`)) + (((c_destchoice_foreign(0.7225565) * FOREIGN) * `6080`) / `184923`)) + (((c_destchoice_age_18_34(3.996802) * AGE_18_34) * `41885`) / `184923`)) + (((c_destchoice_age_35_64(1.517685) * AGE_35_64) * `68638`) / `184923`)) + (((c_destchoice_age_over_65(4.233292) * AGE_OVER_65) * `36249`) / `184923`)) + (`0.008399103276725118` * (((c_destchoice_entscore_18_34(0.5341137) * AGE_18_34) + (c_destchoice_entscore_35_64(2.761052) * AGE_35_64)) + (c_destchoice_entscore_65(2.096477) * AGE_OVER_65)))) + (IN_LF * (c_destchoice_unemp_rate(-2.419098) * `0.03488372093023256`))) + ((((c_destchoice_T34_T34(0

In [34]:
utilities = {}
for puma in puma_recodes.index:
    init = "utilities[{0}] = V{0}".format(puma)
    exec(init)
utilities[0] = V0
V = utilities

In [35]:
av = {}
av[0] = 1
for puma in puma_recodes.index:
    init = "av[{0}] = ALT{0}_available".format(puma)
    exec(init)

In [36]:
av

{0: 1,
 100100: ALT100100_available,
 100200: ALT100200_available,
 100301: ALT100301_available,
 100302: ALT100302_available,
 100400: ALT100400_available,
 100500: ALT100500_available,
 100600: ALT100600_available,
 100700: ALT100700_available,
 100800: ALT100800_available,
 100900: ALT100900_available,
 101000: ALT101000_available,
 101100: ALT101100_available,
 101200: ALT101200_available,
 101301: ALT101301_available,
 101302: ALT101302_available,
 101303: ALT101303_available,
 101304: ALT101304_available,
 101305: ALT101305_available,
 101400: ALT101400_available,
 101500: ALT101500_available,
 101600: ALT101600_available,
 101700: ALT101700_available,
 101800: ALT101800_available,
 101900: ALT101900_available,
 102000: ALT102000_available,
 102100: ALT102100_available,
 102200: ALT102200_available,
 102300: ALT102300_available,
 102400: ALT102400_available,
 102500: ALT102500_available,
 102600: ALT102600_available,
 102701: ALT102701_available,
 102702: ALT102702_available,
 10

In [37]:
import biogeme.messaging as msg
logger = msg.bioMessage()
logger.setDetailed()
logger.allMessages()

'*** Messages from biogeme 3.2.8 [2022-11-29]\n'

In [38]:
# Definition of the model. This is the contribution of each
# observation to the log likelihood function.
# estimating the CHOSEN field
logprob = models.loglogit(V, av, CHOICE)

# formulas = {"loglike": logprob, "weight": W0}

# Create the Biogeme object
biogeme = bio.BIOGEME(database, logprob, suggestScales=True)
biogeme.modelName = 'us_full_availability'

# biogeme.saveIterations = False
# biogeme.loadSavedIteration = False

[08:18:55] < General >   Remove 249 unused variables from the database as only 7047 are used.
[08:19:06] < Detailed >  It is suggested to scale the following variables.
[08:19:06] < Detailed >  Multiply ALT800814_dist by	1e-07 because the largest (abs) value is	3723862.0
[08:19:06] < Detailed >  Multiply ALT3702202_dist by	1e-07 because the largest (abs) value is	4559012.0
[08:19:06] < Detailed >  Multiply ALT5151145_own_job by	0.0001 because the largest (abs) value is	7570.0
[08:19:06] < Detailed >  Multiply ALT2702200_dist by	1e-07 because the largest (abs) value is	3301608.0
[08:19:06] < Detailed >  Multiply ALT2401102_dist by	1e-07 because the largest (abs) value is	4787116.0
[08:19:06] < Detailed >  Multiply ALT4805305_own_job by	0.0001 because the largest (abs) value is	16976.0
[08:19:06] < Detailed >  Multiply ALT3702400_dist by	1e-07 because the largest (abs) value is	4520633.0
[08:19:06] < Detailed >  Multiply ALT800806_own_job by	0.0001 because the largest (abs) value is	6419

: 

: 

In [ ]:
# Calculate the null log likelihood for reporting. (likelihood of predicting every entry's alterantive correctly if alternatives are randomly chosen)
biogeme.calculateNullLoglikelihood(av)
# -1342213.4391806722

-1342213.4391806722

In [ ]:
#estimate parameters
results = biogeme.estimate()
pandasResults = results.getEstimatedParameters()
print(pandasResults)

[10:55:21] < General >   *** Initial values of the parameters are obtained from the file __us_full_final.iter
[10:55:21] < Warning >   Cannot read file __us_full_final.iter. Statement is ignored.
[10:55:43] < Detailed >  Log likelihood (N = 253090):  -159092.9
[10:55:43] < Detailed >  ** Optimization: Newton with trust region for simple bounds
[11:33:36] < General >   Log likelihood (N = 253090):  -159092.9 Gradient norm:      8e+09 Hessian norm:       3e+16 
[11:34:38] < Detailed >  Log likelihood (N = 253090):  -156346.3
[12:10:52] < General >   Log likelihood (N = 253090):  -156346.3 Gradient norm:      2e+09 Hessian norm:       2e+16 
[12:10:52] < Detailed >  1 f= 0.6177499 projected rel. grad.= 7e+03 rel. change=  0.58 delta=     2 rho=   1.1 ++
[12:11:44] < Detailed >  Log likelihood (N = 253090):  -156216.7
[12:47:44] < General >   Log likelihood (N = 253090):  -156216.7 Gradient norm:      1e+08 Hessian norm:       2e+16 
[12:47:44] < Detailed >  2 f= 0.6172377 projected rel. g

In [ ]:
# # occupation-based coefficients

# c_adm = Beta("c_adm", 0, None, None, 0)
# c_agr = Beta("c_agr", 0, None, None, 0)
# c_con = Beta("c_con", 0, None, None, 0)
# c_edu = Beta("c_edu", 0, None, None, 0)
# c_ent = Beta("c_ent", 0, None, None, 0)
# c_ext = Beta("c_ext", 0, None, None, 0)
# c_fin = Beta("c_fin", 0, None, None, 0)
# c_fod = Beta("c_fod", 0, None, None, 0)
# c_inf = Beta("c_inf", 0, None, None, 0)
# c_med = Beta("c_med", 0, None, None, 0)
# c_mfg = Beta("c_mfg", 0, None, None, 0)
# c_mng = Beta("c_mng", 0, None, None, 0)
# c_prf = Beta("c_prf", 0, None, None, 0)
# c_pub = Beta("c_pub", 0, None, None, 0)
# c_rel = Beta("c_rel", 0, None, None, 0)
# c_ret = Beta("c_ret", 0, None, None, 0)
# c_srv = Beta("c_srv", 0, None, None, 0)
# c_trn = Beta("c_trn", 0, None, None, 0)
# c_utl = Beta("c_utl", 0, None, None, 0)
# c_whl = Beta("c_whl", 0, None, None, 0)

In [ ]:
# # Staying Choice Parameters with starting values of 0
# c_stay_married = Beta("c_stay_married", 0, None, None, 0)
# c_stay_age_18_22 = Beta("c_stay_age_18_22", 0, None, None, 0)
# c_stay_age_23_29 = Beta("c_stay_age_23_29", 0, None, None, 0)
# c_stay_age_30_39 = Beta("c_stay_age_30_39", 0, None, None, 0)
# c_stay_age_40_49 = Beta("c_stay_age_40_49", 0, None, None, 1)
# c_stay_age_50_64 = Beta("c_stay_age_50_64", 0, None, None, 0)
# c_stay_age_65 = Beta("c_stay_age_65", 0, None, None, 0)
# c_stay_edu_nohigh = Beta("c_stay_edu_nohigh", 0, None, None, 0)
# c_stay_edu_somecollege = Beta("c_stay_edu_somecollege", 0, None, None, 0)
# c_stay_edu_college = Beta("c_stay_edu_college", 0, None, None, 0)
# c_stay_edu_beyondhs = Beta('c_stay_edu_beyondhs', 0, None, None, 0)
# c_stay_child = Beta("c_stay_child", 0, None, None, 0)
# c_stay_unemployed = Beta("c_stay_unemployed", 0, None, None, 0)
# c_stay = Beta("c_stay", 15, None, None, 0)
# c_stay_foreign = Beta("c_stay_foreign", 0, None, None, 0)
# c_stay_dens = Beta("c_stay_dens", 0, None, None, 0)
# c_stay_college = Beta("c_stay_college", 0, None, None, 0)
# c_stay_rec_child = Beta("c_stay_rec_child", 0, None, None, 0)
# c_stay_rec_mar = Beta("c_stay_rec_mar", 0, None, None, 0)
# c_stay_rec_nomar = Beta("c_stay_rec_nomar", 0, None, None, 0)
# c_stay_mil = Beta("c_stay_mil", 0, None, None, 0)
# c_stay_2work_mar = Beta("c_stay_2work_mar", 0, None, None, 0)
# c_stay_1work_mar = Beta("c_stay_1work_mar", 0, None, None, 0)
# c_stay_otherfamily = Beta("c_stay_otherfamily", 0, None, None, 0)
# c_stay_income = Beta("c_stay_income", 0, None, None, 0)
# c_stay_hhinc = Beta("c_stay_hhinc", 0, None, None, 0)
# c_stay_hurent = Beta('c_stay_hurent', 0, None, None, 0)
# c_stay_vac = Beta('c_stay_vac', 0, None, None, 0)
# c_stay_hhcost = Beta("c_stay_hhcost", 0, None, None, 0)
# c_stay_unemp_rate = Beta("c_stay_unemp_rate", 0, None, None, 0)
# c_stay_hh_val = Beta("c_stay_hh_val", 0, None, None, 0)
# c_stay_partcp = Beta("c_stay_partcp", 0, None, None, 0)
# c_stay_rentcost = Beta("c_stay_rentcost", 0, None, None, 0)
# c_stay_owncost = Beta("c_stay_owncost", 0, None, None, 0)

In [ ]:
# # Bounded staying choice parameters
# c_stay_income = Beta("c_stay_income", 0, 0, None, 0)
# c_stay_hhinc = Beta("c_stay_hhinc", 0, 0, None, 0)
# c_stay_hurent = Beta('c_stay_hurent', 0, None, 0, 0)
# c_stay_vac = Beta('c_stay_vac', 0, None, None, 0)
# c_stay_hhcost = Beta("c_stay_hhcost", 0, None, 0, 0)
# c_stay_unemp_rate = Beta("c_stay_unemp_rate", 0, None, 0, 0)
# c_stay_hh_val = Beta("c_stay_hh_val", 0, 0, None, 0)

In [ ]:
# # Destination Choice Parameters to be estimated
# # Beta(name of the factor, initial value of the coefficient, lower bound, upper bound, whether or not
# c_destchoice_dist=Beta('c_destchoice_dist', 0, None, None, 0)
# c_destchoice_logdist=Beta('c_destchoice_logdist', 0, None, None, 0)
# c_destchoice_unemp_rate = Beta("c_destchoice_unemp", 0, None, None, 0)
# c_destchoice_hhcost = Beta("c_destchoice_hhcost", 0, None, None, 0)
# c_destchoice_college = Beta("c_destchoice_college", 0, None, None, 0)
# c_destchoice_age_18_34 = Beta("c_destchoice_age_18_34", 0, None, None, 0)
# c_destchoice_age_35_64 = Beta("c_destchoice_age_35_64", 0, None, None, 0)
# c_destchoice_age_over_65 = Beta("c_destchoice_age_over_65", 0, None, None, 0)
# c_destchoice_foreign = Beta("c_destchoice_foreign", 0, None, None, 0)
# c_destchoice_pincp = Beta("c_destchoice_pincp", 0, None, None, 0)

# c_destchoice_entscore_18_34 = Beta("c_destchoice_entscore_18_34", 0, None, None, 0)
# c_destchoice_entscore_35_64 = Beta("c_destchoice_entscore_35_64", 0, None, None, 0)
# c_destchoice_entscore_65 = Beta("c_destchoice_entscore_65", 0, None, None, 0)

# c_destchoice_T34_T34 = Beta("c_destchoice_T34_T34", 0, None, None, 1) 
# c_destchoice_T34_Metro = Beta('c_destchoice_T34_Metro', 0, None, None, 0)
# c_destchoice_T34_Micro = Beta("c_destchoice_T34_Micro", 0, None, None, 0)
# c_destchoice_Metro_T34 = Beta("c_destchoice_Metro_T34", 0, None, None, 0)
# c_destchoice_Metro_Metro = Beta('c_destchoice_Metro_Metro', 0, None, None, 0)
# c_destchoice_Metro_Micro = Beta('c_destchoice_Metro_Micro', 0, None, None, 0)
# c_destchoice_Micro_T34 = Beta("c_destchoice_Micro_T34", 0, None, None, 0)
# c_destchoice_Micro_Metro = Beta("c_destchoice_Micro_Metro", 0, None, None, 0)
# c_destchoice_Micro_Micro = Beta("c_destchoice_Micro_Micro", 0, None, None, 0)

# c_destchoice_geo_spec_job = Beta("c_destchoice_geo_spec_job", 0, None, None, 0)
# c_destchoice_high_ed_job = Beta("c_destchoice_high_ed_job", 0, None, None, 0)
# c_destchoice_license_job = Beta("c_destchoice_license_job", 0, None, None, 0)
# c_destchoice_other = Beta("c_destchoice_other", 0, None, None, 0)

# c_destchoice_samestate = Beta("c_destchoice_samestate", 0, None, None, 0)
# c_destchoice_birthstate = Beta("c_destchoice_birthstate", 0, None, None, 0)
# c_destchoice_samecbsa = Beta("c_destchoice_samecbsa", 0, None, None, 0)
# c_destchoice_cbsa_dist = Beta("c_destchoice_cbsa_dist", 0, None, None, 0)
# c_destchoice_military = Beta("c_destchoice_military", 0, None, None, 0)